In [1]:
# Google Colabでkaggle.jsonをアップロード
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kojikojipro","key":"04c6e3329de941099c00e7a9a89a384a"}'}

In [0]:
# Google Colabでkaggle.jsonをフォルダ移動
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# google colabでkaggleのアップデートする
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

# Google Colabでデータセットをダウンロード
!kaggle competitions download -c flower-classification-with-tpus

# ファイルを解凍
!unzip '*.zip'

In [7]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.losses as losses
import tensorflow.keras.metrics as metrics
import tensorflow.keras.optimizers as optimizers
import numpy as np
import os

# tpu用
# 詳細 https://www.tensorflow.org/guide/distributed_training#tpustrategy
tpu_grpc_url = "grpc://" + os.environ["COLAB_TPU_ADDR"]
tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
# TF2.0の場合、ここを追加
tf.config.experimental_connect_to_cluster(tpu_cluster_resolver)
# TF2.0の場合、今後experimentialが取れる可能性がある
tf.tpu.experimental.initialize_tpu_system(tpu_cluster_resolver)
# ここも同様
strategy = tf.distribute.experimental.TPUStrategy(tpu_cluster_resolver)

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

INFO:tensorflow:Initializing the TPU system: 10.84.186.82:8470


INFO:tensorflow:Initializing the TPU system: 10.84.186.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [0]:
def create_model(input_size):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    # モデルの形状を表示
    model.summary()
    return model

In [0]:
with strategy.scope():
    model = create_model()
    loss = losses.SparseCategoricalCrossentropy()
    acc = metrics.SparseCategoricalAccuracy()
    optim = optimizers.Adam()

    # train
    model.compile(optimizer=optim, loss=loss, metrics=[acc])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128,
                steps_per_epoch=X_train.shape[0] // 128, validation_steps=X_test.shape[0] // 128)
    # numpyの場合：ValueError: The number of samples 60000 is not divisible by batch size 128.と怒られるのでstepを指定

    # eval
    val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)
    print(val_loss, val_acc)